In [1]:
%cd ../

/home/hoanghu/projects/Food-Waste-Optimization/experiments_hoangle


In [2]:
import numpy as np
import pandas as pd
from itertools import combinations, product

from utils import Paths

# Load related processed data

In [3]:
wastes = pd.read_excel(Paths.pred_biowaste())

wastes.head()

,meal_id,waste
0,1,0.043050
1,2,0.065768
2,3,0.010000
3,4,0.010000
4,5,0.235866


In [4]:
co2 = pd.read_excel(Paths.pred_co2())

co2.head()

,meal_id,co2
0,1,0.57
1,2,0.64
2,3,0.99
3,4,0.54
4,5,1.28


In [5]:
pieces_whole = pd.read_excel(Paths.pred_n_pcs_whole(), parse_dates=['date'])

pieces_whole.head()

,date,pcs,restaurant
0,2024-09-02,728.91,Chemicum
1,2024-09-03,729.82,Chemicum
2,2024-09-04,707.55,Chemicum
3,2024-09-05,660.46,Chemicum
4,2024-09-06,678.02,Chemicum


In [6]:
pieces_dishes = pd.read_excel(Paths.pred_n_pcs_per_dish(), parse_dates=['date'])

pieces_dishes.head()

,meal_id,date,pcs
0,1,2024-09-02,202
1,2,2024-09-02,76
2,3,2024-09-02,129
3,4,2024-09-02,78
4,5,2024-09-02,94


In [7]:
dim_dishes = pd.read_excel(Paths.dim_dishes(), index_col=None)

dim_dishes.head()

,meal_id,restaurant,category,dish
0,1,Chemicum,chicken,BBQ-Broilerikastiketta
1,2,Chemicum,chicken,Broileria appelsiini-currykastikkeessa
2,3,Chemicum,chicken,Broileria pekonikastikkeessa
3,4,Chemicum,chicken,Broileria pestokastikkeessa
4,5,Chemicum,chicken,Broilerinkoipea


# Create all possible combinations

In [8]:
N_VEGAN_PER_DAY = 2
N_OTHER_PER_DAY = 2

N_LIMIT = 2_000_000     # This threshold limits the number of considering menus

restaurant = "Physicum"
date = pd.to_datetime('2024-09-06')

## Create dataframe legitimate menus (2 vegans per day)

In [9]:
dishes_vegan = dim_dishes.query(f"category == 'vegan' and restaurant == '{restaurant}'")['meal_id'].unique()
dishes_other = dim_dishes.query(f"category != 'vegan' and restaurant == '{restaurant}'")['meal_id'].unique()

In [10]:
list_menus = product(combinations(dishes_vegan, N_VEGAN_PER_DAY), combinations(dishes_other, N_OTHER_PER_DAY))

menus = pd.DataFrame.from_records([
    {
        'dish_1': menu[0][0],
        'dish_2': menu[0][1],
        'dish_3': menu[1][0],
        'dish_4': menu[1][1],
    }
    for menu in list_menus
])

# This line is used to limit the the number of considering menus
menus = menus.sample(min([N_LIMIT, len(menus)])).reset_index(drop=True)

menus.head()

,dish_1,dish_2,dish_3,dish_4
0,344,347,338,339
1,344,348,336,337
2,345,348,351,353
3,347,348,339,351
4,345,346,335,338


## Merge with other tables

In [11]:
# Get 'waste' for all dishes
menus = (
    menus
    .merge(wastes, left_on='dish_1', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'waste': 'waste_1'})
    .merge(wastes, left_on='dish_2', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'waste': 'waste_2'})
    .merge(wastes, left_on='dish_3', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'waste': 'waste_3'})
    .merge(wastes, left_on='dish_4', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'waste': 'waste_4'})
)

# Get 'co2' for all dishes
menus = (
    menus
    .merge(co2, left_on='dish_1', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'co2': 'co2_1'})
    .merge(co2, left_on='dish_2', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'co2': 'co2_2'})
    .merge(co2, left_on='dish_3', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'co2': 'co2_3'})
    .merge(co2, left_on='dish_4', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'co2': 'co2_4'})
)

# Get no. predicted sold pieces for all dishes
pcs_dishes_each = pieces_dishes[pieces_dishes['date'] == date].drop(columns='date')
menus = (
    menus
    .merge(pcs_dishes_each, left_on='dish_1', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'pcs': 'pcs_1'})
    .merge(pcs_dishes_each, left_on='dish_2', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'pcs': 'pcs_2'})
    .merge(pcs_dishes_each, left_on='dish_3', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'pcs': 'pcs_3'})
    .merge(pcs_dishes_each, left_on='dish_4', right_on='meal_id', how='left').drop(columns='meal_id').rename(columns={'pcs': 'pcs_4'})
)

# Filter menus whose CO2 or waste of any dish is missng
menus = menus[~menus.isna().any(axis=1)]
menus.head()

,dish_1,dish_2,dish_3,dish_4,waste_1,waste_2,waste_3,waste_4,co2_1,co2_2,co2_3,co2_4,pcs_1,pcs_2,pcs_3,pcs_4
1,344,348,336,337,0.01,0.010000,0.316988,0.259712,0.55,0.43,0.91,0.84,8,6,5,6
2,345,348,351,353,0.01,0.010000,0.010000,0.442612,0.41,0.43,0.91,1.32,5,6,18,5
5,344,348,334,349,0.01,0.010000,0.010000,0.248939,0.55,0.43,0.91,1.14,8,6,27,6
6,344,347,337,353,0.01,0.023193,0.259712,0.442612,0.55,0.43,0.84,1.32,8,11,6,5
7,345,346,335,336,0.01,0.378258,0.235434,0.316988,0.41,0.57,0.98,0.91,5,12,9,5


## Calculate terms

In [12]:
THETA_CO2 = 0.5
THETA_WASTE = 0.04
ALPHA_PCS = 2
ALPHA_CO2 = 1
ALPHA_WASTE = 1

In [13]:
n_pieces_whole = pieces_whole[(pieces_whole['restaurant'] == restaurant) & (pieces_whole['date'] == date)]['pcs'].item()

menus['total_co2'] = (
    menus['co2_1'] * menus['pcs_1']
    + menus['co2_2'] * menus['pcs_2']
    + menus['co2_3'] * menus['pcs_3']
    + menus['co2_4'] * menus['pcs_4']
)

menus['total_waste'] = (
      menus['waste_1'] * menus['pcs_1']
    + menus['waste_2'] * menus['pcs_2']
    + menus['waste_3'] * menus['pcs_3']
    + menus['waste_4'] * menus['pcs_4']
)

menus['total_pcs_from_dishes'] = menus['pcs_1'] + menus['pcs_2'] + menus['pcs_3'] + menus['pcs_4']

menus['co2_per_customer'] = menus['total_co2'] / menus['total_pcs_from_dishes']
menus['waste_per_customer'] = menus['total_waste'] / menus['total_pcs_from_dishes']

# Calculate fitness value

menus['fitness'] = (
    ALPHA_PCS * np.abs(menus['total_pcs_from_dishes'] / n_pieces_whole - 1) 
    + ALPHA_CO2 * menus['co2_per_customer'] / THETA_CO2 
    + ALPHA_WASTE * menus['waste_per_customer'] / THETA_WASTE
)

# Remove redudant columns
cols = [
    'dish_1',
    'dish_2',
    'dish_3', 
    'dish_4',
    'total_co2', 
    'total_waste',
    'total_pcs_from_dishes',
    'co2_per_customer',
    'waste_per_customer',
    'fitness'
]

menus = menus[cols]

# Sort w.r.t 'fitness'
menus.sort_values(by='fitness', ascending=True, inplace=True)

menus.head()

,dish_1,dish_2,dish_3,dish_4,total_co2,total_waste,total_pcs_from_dishes,co2_per_customer,waste_per_customer,fitness
956,344,347,334,351,50.08,0.785127,64,0.782500,0.012268,2.596790
142,347,348,334,351,48.26,0.765127,62,0.778387,0.012341,2.630234
1335,345,347,334,351,47.73,0.755127,61,0.782459,0.012379,2.659257
1142,344,348,334,351,47.93,0.590000,59,0.812373,0.010000,2.699447
434,344,347,334,342,42.80,0.705127,56,0.764286,0.012592,2.727823
